In [49]:
import pandas as pd
import numpy as np

from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split, StratifiedKFold

img_size_ori = 101

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 20 <= i :
            return i

train_df = pd.read_csv("./input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("./input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)

train_df["masks"] = [np.array(load_img("./input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in (train_df.index)]
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2) / 3
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=57)

train_df["fold"] = -1

fold = 0
for train_index, valid_index in skf.split(train_df.index, train_df.coverage_class):
    train_df.loc[train_df.index[valid_index],["fold"]]=fold
    fold+=1
    
train_df.sort_index(inplace=True)
train_df.to_csv("folds_stratified_by_coverage.csv", columns=["fold"])

train_df.fold.value_counts()

0    671
1    669
2    667
3    665
4    665
5    663
Name: fold, dtype: int64

In [50]:
train_df.coverage_class.value_counts()

0    1562
1     747
2     349
4     325
3     291
5     251
7     245
6     230
Name: coverage_class, dtype: int64

In [51]:
671/(4000-671)

0.20156203063983177